# Passport Detection Project - Model Training

**Student:** [Your Name]  
**Project:** Fake Passport Detection Using CNN and Rule-Based Validation  
**Purpose:** Educational demonstration of computer vision and deep learning

---

## 📚 What This Notebook Does

1. Load and preprocess passport images
2. Handle class imbalance
3. Build CNN model using transfer learning
4. Train the model
5. Evaluate performance
6. Save trained model

---

## ⚠️ IMPORTANT NOTES

- Make sure you have dataset in correct folder structure
- Training may take 30-60 minutes depending on your hardware
- Model will be saved to `models/passport_cnn.h5`

---

## Step 1: Import Required Libraries

In [ ]:
# Basic imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Deep learning
import tensorflow as tf
from tensorflow import keras

# Our custom modules
sys.path.append('..')  # Add parent directory to path
from src.data.preprocessing import create_data_generators, calculate_class_weights, visualize_samples
from src.models.cnn_model import create_cnn_model, train_model, plot_training_history

# Print TensorFlow version
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ Keras version: {keras.__version__}")

# Check if GPU is available (optional, makes training faster)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ GPU available: {len(gpus)} device(s)")
else:
    print("⚠ No GPU detected, training will use CPU (slower)")

## Step 2: Set Up Project Paths

Make sure your dataset is organized like this:

```
data/
├── train/
│   ├── real/
│   └── fake/
├── validation/
│   ├── real/
│   └── fake/
└── test/
    ├── real/
    └── fake/
```

In [ ]:
# Define paths
BASE_DIR = os.path.abspath('..')
DATA_DIR = os.path.join(BASE_DIR, 'data')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VAL_DIR = os.path.join(DATA_DIR, 'validation')
TEST_DIR = os.path.join(DATA_DIR, 'test')
MODEL_DIR = os.path.join(BASE_DIR, 'models')

# Create models directory if it doesn't exist
os.makedirs(MODEL_DIR, exist_ok=True)

# Check if directories exist
print("Checking data directories...")
print(f"Train directory: {TRAIN_DIR} - {'✓ Exists' if os.path.exists(TRAIN_DIR) else '✗ NOT FOUND'}")
print(f"Validation directory: {VAL_DIR} - {'✓ Exists' if os.path.exists(VAL_DIR) else '✗ NOT FOUND'}")
print(f"Test directory: {TEST_DIR} - {'✓ Exists' if os.path.exists(TEST_DIR) else '✗ NOT FOUND'}")

## Step 3: Create Data Generators

Data generators:
- Load images in batches (saves memory)
- Apply data augmentation to training data
- Normalize pixel values (0-1)

**What is data augmentation?**  
Creating slightly modified versions of images (rotation, brightness, etc.) to increase dataset variety and prevent overfitting.

In [ ]:
print("Creating data generators...\n")

# Create generators
train_generator, validation_generator = create_data_generators(TRAIN_DIR, VAL_DIR)

# Display info
print(f"\n{'='*60}")
print("DATA GENERATOR SUMMARY")
print(f"{'='*60}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Batch size: {train_generator.batch_size}")
print(f"Classes: {train_generator.class_indices}")
print(f"{'='*60}")

## Step 4: Visualize Sample Images

Let's look at some sample images to verify:
1. Images are loading correctly
2. Data augmentation is working
3. Labels are correct

In [ ]:
print("Displaying sample training images with augmentation...\n")
visualize_samples(train_generator, num_samples=9)

## Step 5: Calculate Class Weights

**Why do we need class weights?**

In real scenarios, we usually have:  
- More REAL passport images
- Fewer FAKE passport images

This is called **class imbalance**.

**Problem:**  
Model might learn to just predict "REAL" all the time  
(Example: With 300 real and 100 fake, always predicting REAL gives 75% accuracy!)

**Solution:**  
Give more weight to the minority class (fake)  
This forces the model to learn both classes equally.

In [ ]:
# Calculate class weights
class_weights = calculate_class_weights(train_generator)

print("\nClass weights will be used during training to handle imbalance.")

## Step 6: Create CNN Model

**What is a CNN?**  
Convolutional Neural Network - designed for image analysis.  
Learns visual patterns automatically (edges → textures → objects)

**What is transfer learning?**  
Using a model already trained on millions of images (ImageNet)  
Then fine-tuning it for our specific task (passport detection)

**Our architecture:**
1. **Base:** EfficientNetB0 (pre-trained)
2. **Custom head:** Dense layers for passport classification
3. **Output:** Sigmoid (probability 0-1)

In [ ]:
# Create model
model = create_cnn_model()

print("\n✓ Model created successfully!")
print(f"✓ Total parameters: {model.count_params():,}")

## Step 7: Train the Model

**What happens during training?**

1. Model looks at images
2. Makes predictions
3. Calculates how wrong it was (loss)
4. Updates weights to improve
5. Repeats for all images = 1 epoch
6. Returns multiple epochs

**Callbacks used:**
- **Early Stopping:** Stop if no improvement
- **Reduce LR:** Lower learning rate if stuck
- **Model Checkpoint:** Save best model

**This will take 30-60 minutes**  
Go grab a coffee! ☕

In [ ]:
# Train model
history = train_model(
    model=model,
    train_generator=train_generator,
    validation_generator=validation_generator,
    class_weights=class_weights,
    epochs=20  # Will stop early if no improvement
)

## Step 8: Visualize Training History

**What to look for:**

**GOOD SIGNS:**
- Both training and validation accuracy increase
- Both training and validation loss decrease
- Curves are close to each other
- Curves stabilize at the end

**BAD SIGNS (overfitting):**
- Training keeps improving, but validation stops or gets worse
- Large gap between training and validation
- Validation curve is very noisy

**If overfitting occurs:**
- Add more data augmentation
- Increase dropout rate
- Collect more training data

In [ ]:
# Plot training curves
plot_training_history(history)

## Step 9: Evaluate on Test Set

**Why test set?**  
To evaluate model on completely unseen data  
This simulates real-world performance

**Metrics:**
- **Accuracy:** Overall correctness
- **Precision:** When model says FAKE, how often is it right?
- **Recall:** Of all actual fakes, how many did we catch?
- **F1-Score:** Balance between precision and recall

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create test generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

# Evaluate
print("Evaluating on test set...\n")
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"\n{'='*60}")
print("TEST SET RESULTS")
print(f"{'='*60}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'='*60}")

## Step 10: Generate Confusion Matrix

**What is a confusion matrix?**  
Shows where the model makes mistakes:

```
                Predicted
              FAKE    REAL
Actual FAKE    TP      FN
       REAL    FP      TN
```

- **TP (True Positive):** Correctly identified fakes
- **TN (True Negative):** Correctly identified reals
- **FP (False Positive):** Real marked as fake (bad!)
- **FN (False Negative):** Fake marked as real (very bad!)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()
true_classes = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['FAKE', 'REAL'],
            yticklabels=['FAKE', 'REAL'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
print("\nDetailed Metrics:")
print("="*60)
target_names = ['FAKE', 'REAL']
print(classification_report(true_classes, predicted_classes, target_names=target_names))

## Step 11: Save the Model

Save the trained model so we can use it later without retraining.

In [ ]:
# Model is already saved by ModelCheckpoint callback
# But let's also save the final version

final_model_path = os.path.join(MODEL_DIR, 'passport_cnn_final.h5')
model.save(final_model_path)

print(f"✓ Model saved to: {final_model_path}")
print(f"✓ Best model saved to: {os.path.join(MODEL_DIR, 'passport_cnn.h5')}")
print("\n✅ Training complete! You can now use the model for predictions.")

## 🎯 Summary

**What we accomplished:**
1. ✓ Loaded and preprocessed passport images
2. ✓ Applied data augmentation
3. ✓ Handled class imbalance with weights
4. ✓ Built CNN model using transfer learning
5. ✓ Trained model with callbacks
6. ✓ Evaluated performance on test set
7. ✓ Generated confusion matrix
8. ✓ Saved trained model

**Next steps:**
- Use Grad-CAM to visualize model attention
- Implement rule-based validation
- Create hybrid decision system
- Build web interface with Streamlit

---

**For your report/presentation:**

*"I implemented a CNN-based passport verification system using transfer learning with EfficientNetB0. The model achieved [X]% accuracy on the test set. I addressed class imbalance using weighted loss and prevented overfitting through data augmentation and dropout. The system successfully combines deep learning with rule-based validation for improved robustness and interpretability."*